In [ ]:
#| default_exp structure

# structure

> Basic functionality related to the structure of the (XML) data

In [ ]:
#| export
import pathlib
import re

import numpy as np
import pandas as pd

For testing purposes

In [ ]:
import dlsproc.xml

## Sample data

Directory where the data (*XML* files) are stored

In [ ]:
directory = pathlib.Path.cwd().parent / 'samples'
assert directory.exists()
directory

PosixPath('/usr/local/bak/dlsproc/samples')

A (sample) file in that directory

In [ ]:
xml_file = directory / 'PlataformasAgregadasSinMenores_20220104_030016_1.atom'
assert xml_file.exists()
xml_file

PosixPath('/usr/local/bak/dlsproc/samples/PlataformasAgregadasSinMenores_20220104_030016_1.atom')

In [ ]:
df = dlsproc.xml.to_df(xml_file)
df

,id,summary,title,updated,ContractFolderStatus - ContractFolderID,ContractFolderStatus - ContractFolderStatusCode,ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID,ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - ProcurementProject - Name,...,ContractFolderStatus - LegalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - TechnicalDocumentReference - ID,ContractFolderStatus - TechnicalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - ProcurementProject - PlannedPeriod - StartDate,ContractFolderStatus - ProcurementProject - PlannedPeriod - EndDate,ContractFolderStatus - LocatedContractingParty - Party - PartyIdentification - ID,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndDate,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndTime,ContractFolderStatus - TenderResult - AwardedTenderedProject - ProcurementProjectLotID
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: C. 2-2021; Órgano de Contrataci...,L'objecte del contracte és la renovació de tot...,2022-01-03T01:11:41.826+01:00,C. 2-2021,ADJ,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Sant Ramon,Entitats municipals de Catalunya,L'objecte del contracte és la renovació de tot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8128_3/2021; Órgano de Contrata...,Obras de restauración hidromorfológica del río...,2022-01-03T01:00:11.194+01:00,8128_3/2021,PUB,NaN,Pleno del Ayuntamiento,AYUNTAMIENTO DE MONREAL,Obras de restauración hidromorfológica del río...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1000_0005-CP01-2021-000063; Órg...,Contrato del servicio de realización de labore...,2022-01-03T01:00:10.399+01:00,1000_0005-CP01-2021-000063,EV,NaN,El Director General de Comunicación y Relacion...,"Departamento de Presidencia, Igualdad, Función...",Contrato del servicio de realización de labore...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1379/2020 4738; Órgano de Contr...,Obres de renovació de l'enllumenat públic a la...,2022-01-03T00:11:40.740+01:00,1379/2020 4738,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Canet de Mar,Entitats municipals de Catalunya,Obres de renovació de l'enllumenat públic a la...,...,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 2021-44; Órgano de Contratación...,Subministre i la instal·lació fotovoltaica en ...,2022-01-03T00:11:40.696+01:00,2021-44,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Valls,Entitats municipals de Catalunya,Subministre i la instal·lació fotovoltaica en ...,...,https://contractaciopublica.gencat.cat/ecofin_...,Enllac plec clausules tecniques.doc,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1005_391-2021; Órgano de Contra...,Apoyo a la gestión del patrimonio filmográfico...,2021-12-31T01:00:14.946+01:00,1005_391-2021,PUB,NaN,Dirección General de Cultura-Institución Prínc...,"Departamento de Cultura, Deporte y Juventud",Apoyo a la gestión del patrimonio filmográfico...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8165_3/2021; Órgano de Contrata...,Asistencia técnica para la prestación del serv...,2021-12-31T01:00:14.393+01:00,

## Nested tags

When a *tag* is nested inside other *tag* the string below is used to assemble the name of the resulting *column* from those of the original tags

In [ ]:
#| export
nested_tags_separator = ' - '

For instance, a *tag* `<month>` inside a *tag* `<date>` will yield a *column*

In [ ]:
f'month{nested_tags_separator}date'

'month - date'

A high-level function to do exactly that

In [ ]:
#| export
def assemble_name(tags: list) -> str:
    """
    Assemble the name of field/column in the DataFrame from a path of nested tags.
    
    **Parameters**
    
    - tags: list
    
        List of tags.
    
    **Returns**
    
    - out: str
    
        A suitable name.
    
    """
    
    tags = filter(pd.notna, tags)
    tags = filter(lambda x: x!='', tags)
    
    return nested_tags_separator.join(tags)

In [ ]:
assemble_name(['foo', 'fa'])

'foo - fa'

`NaN`s are ignored...

In [ ]:
assemble_name(['foo', np.nan])

'foo'

...and so are empty strings

In [ ]:
assemble_name(['foo', np.nan, ''])

'foo'

Mutivalued data

`list`s of values are possible. A function to check whether a `pd.Series` (a *column* in a `pd.DataFrame`) has `list`s.

In [ ]:
#| export
def is_multivalued(s: pd.Series) -> bool:
    """
    Returns `True` is the given `pd.Series` has a `list` at any index.
    
    **Parameters**
    
    - s: pandas series
    
        Input
        
    **Returns**
    
    - out: bool
    
        `True` if the input contains some `list`.
    
    """
    
    # return s.apply(lambda x: type(x) == list).any()
    return s.apply(lambda x: (type(x) == list) or (type(x) == np.ndarray)).any()

In [ ]:
multivalued_column = assemble_name(['ContractFolderStatus', 'ValidNoticeInfo', 'NoticeTypeCode'])
is_multivalued(df[multivalued_column])

True

In [ ]:
df[multivalued_column].head()

0    [DOC_CN, DOC_CAN_ADJ]
1                   DOC_CN
2                   DOC_CN
3                   DOC_CN
4                   DOC_CN
Name: ContractFolderStatus - ValidNoticeInfo - NoticeTypeCode, dtype: object

In [ ]:
is_multivalued(df['id'])

False

A function exploiting the above function to return (only) the columns that are multivalued.

In [ ]:
#| export
def multivalued_columns(df: pd.DataFrame) -> list:
    
    # `True` or `False` for every column
    is_column_multivalued = df.apply(is_multivalued, axis='index')
    
    return is_column_multivalued[is_column_multivalued==True].index.tolist()

In [ ]:
multivalued_columns(df)

['ContractFolderStatus - ProcurementProject - RequiredCommodityClassification - ItemClassificationCode',
 'ContractFolderStatus - TenderResult - ResultCode',
 'ContractFolderStatus - TenderResult - ReceivedTenderQuantity',
 'ContractFolderStatus - TenderResult - WinningParty - PartyIdentification - ID',
 'ContractFolderStatus - TenderResult - WinningParty - PartyName - Name',
 'ContractFolderStatus - TenderResult - AwardedTenderedProject - LegalMonetaryTotal - TaxExclusiveAmount',
 'ContractFolderStatus - ValidNoticeInfo - NoticeTypeCode',
 'ContractFolderStatus - ValidNoticeInfo - AdditionalPublicationStatus - PublicationMediaName',
 'ContractFolderStatus - ValidNoticeInfo - AdditionalPublicationStatus - AdditionalPublicationDocumentReference - IssueDate',
 'ContractFolderStatus - TenderResult - AwardedTenderedProject - ProcurementProjectLotID']

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('05_structure.ipynb')